In [1]:
import os
import gym
import gym_donkeycar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import pickle
import birds_eye_vector_space
import random
from pandas import Series, DataFrame
from collections import deque

#from keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from keras.initializers import normal, identity
#from keras.models import model_from_json
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
#from keras import backend as K
#import tensorflow.keras.backend as K

C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find d

In [2]:
# use gpu
#K.tensorflow_backend._get_available_gpus()
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [7]:
# run on the second GPU
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
len(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9053465812462338940
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9401860096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14054652856522372114
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:49:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6271991808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13751037547871604358
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:21:00.0, compute capability: 7.5"
 xla_global_id: 2144165316]

In [9]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Supress scientific notation like: e*+03
np.set_printoptions(suppress=True)

In [12]:
ANCHOR_POINTS = 8
ROAD_ROI = np.array([(120,90),(200,90),(0,200),(320,200)],dtype='float32')
WARPED_IMAGE_SHAPE = np.array([[10,320],[0,0],[200,0],[200,310]],np.int32)            # NEW IMAGE Shape after Warping !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WARPED_IMAGE_HEIGHT = 320
WARPED_IMAGE_WIDTH = 200
NUMBER_OF_DEPTH_LAYERS = 50

VECTOR_SPACE_IMAGE_ROWS = 50
VECTOR_SPACE_IMAGE_COLUMNS = 100
VECTOR_SPACE_IMAGE_CHANNELS = 4 # 4*(50,100) stacked frames 

In [13]:
# function to save the plots
def save_plot(data, name, episode):
    plt.figure(figsize=(8,5), frameon=True)
    #plt.plot([episode for episode in range(len(data))], data)
    plt.plot([ep for ep in range(episode)], data)
    plt.xlabel('Episodes')
    plt.ylabel(name)
    figplot = '%s-episode-%s.png' % (name,episode)
    #figplot = '{}-episode-{}.png'.format(name,episode)
    #location = '{}/saves/torch/{}'.format(os.getcwd(),figplot)
    #location = '%s/collected_data/plots/%s' % (os.getcwd(),figplot)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\plots\\%s' % figplot 
    plt.savefig(location, transparent=False)
    plt.close()

In [14]:
def save_data_as_dataframe(episode_number, reward, loss, measured_time, max_cte, average_cte, last_lap_time):
    episodes = [i for i in range(1, episode_number+1)]
    collected_data = {'episode': episodes, 'reward': reward, 'loss':loss, 'time': measured_time, 'max_cte': max_cte, 'average_cte': average_cte, 'lap_time': last_lap_time}
    df_data = DataFrame.from_dict(collected_data).set_index('episode')
    
    df_name = 'data-episode-%s.pkl' % episode
    #location = '%s/collected_data/raw_data/%s' % (os.getcwd(),df_name)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\raw_data\\%s' % df_name
    df_data.to_pickle(location)

In [15]:
vector_space = birds_eye_vector_space.Vector_space(ANCHOR_POINTS, ROAD_ROI, WARPED_IMAGE_SHAPE, WARPED_IMAGE_HEIGHT, WARPED_IMAGE_WIDTH, NUMBER_OF_DEPTH_LAYERS)

vector space is initialized
loading camera parameters


In [16]:
os.getcwd()

'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\\docker\\src'

In [17]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.t = 0
        self.max_Q = 0
        self.train = True
        # Set to True to train on images with segmented lane lines
        self.lane_detection = False
        
        # Huber loss
        self.huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        # Get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # CNN Input
        #self.input_shape = (VECTOR_SPACE_IMAGE_CHANNELS, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, 4)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        self.input_shape = (1, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        
        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        if (self.train):
            self.epsilon = 1.0
            self.initial_epsilon = 1.0
        else:
            self.epsilon = 1e-6
            self.initial_epsilon = 1e-6
        self.epsilon_min = 0.02
        self.batch_size = 512
        self.train_start = 100
        self.explore = 10000
        
        # Create replay memory using deque
        self.memory = deque(maxlen=10000)
        
        # Create main model and target model                # Double DQN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.model = self.build_model()
        self.target_model = self.build_model()

        # Copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()
        
    def build_model_old(self):
        print('delte this')
        #model = Sequential()
        #model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(VECTOR_SPACE_IMAGE_ROWS,VECTOR_SPACE_IMAGE_COLUMNS,VECTOR_SPACE_IMAGE_CHANNELS)))  #80*80*4
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Flatten())
        #model.add(Dense(512))
        #model.add(Activation('relu'))

        ## 15 categorical bins for Steering angles
        #model.add(Dense(15, activation="linear")) 

        #adam = Adam(lr=self.learning_rate)
        #model.compile(loss='mse',optimizer=adam)

        #return model

    def build_model(self):
        model = keras.Sequential([
            layers.Conv2D(filters=32, strides=(4, 4),kernel_size=8, padding='same', activation='relu', input_shape=(self.input_shape[1:])),
            layers.Conv2D(filters=64, strides=(2, 2),kernel_size=4, padding='same', activation='relu'),
            layers.Conv2D(filters=64, strides=(1, 1),kernel_size=3, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(units=512,activation='relu'),
            layers.Dense(units=15,activation='linear'),
        ])
        #optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        optimizer = tf.keras.optimizers.Adam()
        model.compile(loss='mse', optimizer=optimizer)
        return model
    
    def calculate_loss(self, y_true, y_pred):    
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Get action from model using epsilon-greedy policy
    def get_action(self, s_t):
        if np.random.rand() <= self.epsilon:
            #print("Return Random Value")
            #return random.randrange(self.action_size)
            return np.random.uniform(-1,1)
        else:
            #print("Return Max Q Prediction")
            q_value = self.model.predict(s_t)
            # Convert q array to steering value
            return linear_unbin(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        #if self.epsilon > self.epsilon_min:
        #    #self.epsilon *= self.epsilon_decay
        #    self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore

    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        
        # Epsilion decay over time/amount of trainings
        if self.epsilon >= self.epsilon_min:
            #self.epsilon *= self.epsilon_decay
            #self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore
            #self.epsilon -= 0.00025
            self.epsilon -= 0.00035
            
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
        #print('Starting training\nstate_t: %s\naction_t: %s\nreward_t: %s\nstate_1: %s\nterminal: %s' % (len(state_t), len(action_t), len(reward_t), len(state_t1), len(terminal)))
        state_t = np.concatenate(state_t)
        state_t1 = np.concatenate(state_t1)
        #print('Concat:\nstate: %s\nstate_t1: %s' % (state_t, state_t1))
        targets = self.model.predict(state_t)
        self.max_Q = np.max(targets[0])
        target_val = self.model.predict(state_t1)
        target_val_ = self.target_model.predict(state_t1)
        for i in range(batch_size):
            if terminal[i]:
                targets[i][action_t[i]] = reward_t[i]
            else:
                a = np.argmax(target_val[i])
                targets[i][action_t[i]] = reward_t[i] + self.discount_factor * (target_val_[i][a])

        
        # train on the current batch and return a dictionary with the loss and so on
        #metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True)           
        metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True) #reset Metrix different
        return metrics['loss']
        
    def load_model(self, name):
        self.model.load_weights(name)

    # Save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [18]:
def linear_bin(a):
    """
    Convert a value to a categorical array.

    Parameters
    ----------
    a : int or float
        A value between -1 and 1

    Returns
    -------
    list of int
        A list of length 15 with one item set to 1, which represents the linear value, and all other items set to 0.
    """
    a = a + 1
    b = round(a / (2 / 14))
    arr = np.zeros(15)
    arr[int(b)] = 1
    return arr

In [19]:
def linear_unbin(arr):
    """
    Convert a categorical array to value.

    See Also
    --------
    linear_bin
    """
    if not len(arr) == 15:
        raise ValueError('Illegal array length, must be 15')
    b = np.argmax(arr)
    a = b * (2 / 14) - 1
    return a

## Main training loop

In [20]:
#%% SET UP ENVIRONMENT
# Normal one 
#os.environ['DONKEY_SIM_PATH'] = "/home/zamy/masterthesis/DonkeySimLinux/donkey_sim.x86_64"
os.environ['DONKEY_SIM_PATH'] = "C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\test\\DonkeySimWin\\donkey_sim.exe"
os.environ['DONKEY_SIM_PORT'] = str(9091)
os.environ['DONKEY_SIM_HEADLESS'] = str(0) # "1" is headless

CAMERA_CONF = {'cam_config':{'img_w': '320',
                        'img_h': '200',
                        'img_d': '1',   # 3 for colored Tensor image
                        'img_enc': 'PNG', 
                        'fov': '90', 
                        'fish_eye_x': '0.0', 
                        'fish_eye_y': '0.0', 
                        'offset_x': '0.0', 
                        'offset_y': '0.0', 
                        'offset_z': '0.0', 
                        'rot_x': '0'}}
# Other tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF)

# TRACKS TRACKS - TKarmer Tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF) #,conf=config
# Random track, but the reward is kinda not working, IF THE REWARD WORKS IT IS KINDA RANDOM !!, this maybe debends on the environment
env = gym.make("donkey-generated-roads-v0",conf=CAMERA_CONF) #,conf=config

# Mini Monaco Track
#env = gym.make("donkey-minimonaco-track-v0",conf=CAMERA_CONF) #,conf=config
#env = gym.make("donkey-generated-track-v0") #,conf=config
#env = gym.wrappers.ResizeObservation(env,(200,320))

## DELETE LATER
#env.frameskip = 1
#gym.wrappers.max

# Create DQN Model
# Get size of state and action from environment
state_size = (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)
action_size = 15 # Steering and Throttle
throttle = 0.15 # Set the throttle as a constant value
agent = DQNAgent(state_size, action_size)


# Arrays for data collection / exploratory data analysis
average_cte = []
average_speed = []
measured_time = []
average_rewards = []
average_loss = []
collected_max_cte = []
collected_lap_time = []

# when a model and the data should be saved
save_state = 200

for episode in range(1,5001):
    # Resetting the environment and preprocessing the first image
    observation = env.reset()
    # Graykonvertion of the observation image
    observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
    observation = vector_space.image_preprocessing(observation)
    # frame stacking (4 times)
    obv_stack = np.stack((observation, observation, observation, observation), axis= 2)
    # reshaping for keras
    obv_stack = obv_stack.reshape(1, obv_stack.shape[0], obv_stack.shape[1], obv_stack.shape[2])
    
    #action = np.array([0,0.10]) # drive straight with small speed
    #action = np.array([1,0.1]) # drive straight with small speed
    
    # summed up values for data collection
    # cumulative values for data collection
    total_cte = float(0)
    total_speed = float(0)
    total_reward = float(0)
    total_time = time.time()
    total_loss = float(0)
    max_cte = int(0)
    lap_time = int(0)
        
    # counting the amount of frames per episode
    frames = int(0)
    
    # boolean that describes if the env is done with this episode
    done = False
    while not done:
        # incrementing the amount of frames per episode
        frames += 1
        
        # Making a prediction for the current state
        # and getting the information form the next step
        steering = agent.get_action(obv_stack)
        action = [steering, throttle]
        next_observation, reward, done, info = env.step(action)
        
        current_cte = abs(info['cte'])
        # punish and reward for a high or low 'CTE'
        if current_cte < .25:
            reward += abs(np.log(.10)) * 10
        elif current_cte < .5 and current_cte >= .25:
            reward += abs(np.log(current_cte)) * 10
        elif current_cte < 1:
            reward -= abs(np.log(current_cte))
        else:
            reward -= abs(np.log(current_cte)) * 10
            
        #if the car leaves the track punish it
        if done:
            reward -= 500
        
        # Graykonvertion of the observation image
        next_observation = cv.cvtColor(next_observation, cv.COLOR_BGR2GRAY)
        
        # preprocessing the new observation
        next_observation_show = vector_space.image_preprocessing(next_observation)
        # reshaping for keras
        #next_observation = next_observation_show.reshape(1, next_observation_show.shape[0], next_observation_show.shape[1], 1)
        # appending to the observation stack
        #obv_stack = np.append(next_observation, obv_stack[:, :, :, :3], axis=3)
        
        obv_stack_t1 = np.stack([next_observation_show, obv_stack[0,:,:,0],obv_stack[0,:,:,1],obv_stack[0,:,:,2]], axis=2)
        obv_stack_t1 = obv_stack_t1.reshape(1, obv_stack_t1.shape[0], obv_stack_t1.shape[1], obv_stack_t1.shape[2])
                
        # saving the sample <s, a, r, s'> to the replay memory
        agent.replay_memory(obv_stack, np.argmax(linear_bin(steering)), reward, obv_stack_t1, done)        
        
        # training the DDQN, if training is enabled
        #if agent.train:
        #    loss = agent.train_replay()
        #    if loss != None:
        #        total_loss += loss
        #        #print('loss: %s ' % (total_loss / frames))
        
        # overwriting the stack and incrementing the time/frame counter
        obv_stack = obv_stack_t1
        agent.t += 1       
        
        # adding up the collected data
        current_cte = info['cte'] 
        total_cte += current_cte
        total_speed += info['speed']
        total_reward += reward
        # update if there is any change in incrementation 
        if max_cte < current_cte:
            max_cte = current_cte
        if lap_time < info['last_lap_time']:
            lap_time = info['last_lap_time']
            
        cv.imshow('vec img', next_observation_show)

        # Graykonvertion of the observation image
        #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
        
        # Image preprocessing
        #vec_image = vector_space.image_preprocessing(observation)
        
        #print('REWARD: %s' % reward)
        
        #cv.imshow('vec img', vec_image)
        
        #break       # TODO: DELTE LATER
        if cv.waitKey(25) & 0xFF == ord('q'):
            cv.destroyAllWindows()
            break
    
    # Training after each episode
    if agent.train:    
        total_loss = agent.train_replay()
        #if loss != None:
        #    total_loss = loss
    
    
    # AFTER this Episode & the environment returns True for the done variable
    # updating the target DDQN
    agent.update_target_model()
    
    # calculating the average loss in this episode
    #episode_loss = total_loss.copy() #/ frames
    episode_cte = total_cte / frames
    episode_reward = total_reward / frames
    
    # updating the time parameter
    total_time = abs(total_time - time.time())
    
    # appending the collected data
    average_cte.append(episode_cte)
    average_speed.append(total_speed / frames)
    measured_time.append(total_time)
    average_rewards.append(episode_reward)
    average_loss.append(total_loss)
    collected_max_cte.append(max_cte)
    collected_lap_time.append(lap_time) 
    
    # Testing of saving the plot
    #if episode % 3 == 0 and episode != 0:
    #    save_plot(colleted_rewards, 'Reward', episode)
    #    cv.destroyAllWindows()
    #    break
    
    # Print episode information    
    print('EPISODE: %s | TIME: %s s | REWARD: %s | FRAMES: %s | QMAX: %s | EPSILON: %s | CTE: %s | LOSS: %s' % (episode, total_time, episode_reward, frames, str(agent.max_Q), agent.epsilon, episode_cte, total_loss))    
    
    if episode % save_state == 0 and episode != 0:
        print('saving after %s episodes' % save_state)
        # saving the model
        agent.save_model('C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\models\\model_episode_%s.h5' % episode)        
        
        # Saving the data as plots
        save_plot(average_rewards, 'Reward', episode)
        save_plot(average_cte, 'Average cte', episode)
        save_plot(collected_max_cte, 'Max cte', episode)
        save_plot(measured_time, 'Time', episode)
        save_plot(average_loss, 'Loss', episode)
        save_plot(collected_lap_time, 'Lap Time', episode)
        
        # Saving the collected data
        save_data_as_dataframe(episode, average_rewards, average_loss, measured_time, collected_max_cte, average_cte, collected_lap_time)
    
    # closing all cv windows
    cv.destroyAllWindows()

# closing all cv windows
cv.destroyAllWindows()
# Close the enviroment
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 5.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: port 9091
loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:done sending cam config. {'img_w': '320', 'img_h': '200', 'img_d': '1', 'img_enc': 'PNG', 'fov': '90', 'fish_eye_x': '0.0', 'fish_eye_y': '0.0', 'offset_x': '0.0', 'offset_y': '0.0', 'offset_z': '0.0', 'rot_x': '0'}


EPISODE: 1 | TIME: 23.542293071746826 s | REWARD: 3.6845756607336324 | FRAMES: 218 | QMAX: 7.108527 | EPSILON: 0.99965 | CTE: 1.4987867673810549 | LOSS: 108.2032699584961
EPISODE: 2 | TIME: 11.859683513641357 s | REWARD: 1.4664762468968093 | FRAMES: 210 | QMAX: 21.064056 | EPSILON: 0.9993000000000001 | CTE: 1.4423253411319048 | LOSS: 112.046630859375
EPISODE: 3 | TIME: 14.772543907165527 s | REWARD: 12.37454066016923 | FRAMES: 262 | QMAX: 86.93225 | EPSILON: 0.9989500000000001 | CTE: 0.9097171275099237 | LOSS: 208.02381896972656
EPISODE: 4 | TIME: 15.086402893066406 s | REWARD: 5.381832161040821 | FRAMES: 287 | QMAX: 228.13121 | EPSILON: 0.9986000000000002 | CTE: 1.4258765406564904 | LOSS: 319.6664123535156
EPISODE: 5 | TIME: 8.349722146987915 s | REWARD: 5.884076833604316 | FRAMES: 153 | QMAX: 348.60626 | EPSILON: 0.9982500000000002 | CTE: 1.1872859544183005 | LOSS: 675.665771484375
EPISODE: 6 | TIME: 8.704936981201172 s | REWARD: 4.483664315947461 | FRAMES: 160 | QMAX: 501.13806 | EP

EPISODE: 48 | TIME: 9.449760437011719 s | REWARD: 5.230661035146511 | FRAMES: 174 | QMAX: 51365124.0 | EPSILON: 0.9832000000000019 | CTE: 1.277597414708046 | LOSS: 290264023040.0
EPISODE: 49 | TIME: 8.485709428787231 s | REWARD: 1.3469640360165698 | FRAMES: 154 | QMAX: 43376.695 | EPSILON: 0.9828500000000019 | CTE: 1.5710187083081817 | LOSS: 372950401024.0
EPISODE: 50 | TIME: 8.867529153823853 s | REWARD: 7.125332960328813 | FRAMES: 156 | QMAX: 2538022.0 | EPSILON: 0.9825000000000019 | CTE: 1.045779740301923 | LOSS: 406919839744.0
EPISODE: 51 | TIME: 22.018653869628906 s | REWARD: -7.849034246640376 | FRAMES: 421 | QMAX: 62331710.0 | EPSILON: 0.982150000000002 | CTE: 3.0878133183636556 | LOSS: 569673646080.0
EPISODE: 52 | TIME: 10.742483377456665 s | REWARD: -0.13855675122859523 | FRAMES: 192 | QMAX: 3093627.0 | EPSILON: 0.981800000000002 | CTE: 1.7236285638666669 | LOSS: 657886937088.0
EPISODE: 53 | TIME: 12.550617218017578 s | REWARD: 4.326613385605693 | FRAMES: 232 | QMAX: 101612980

...

EPISODE: 1260 | TIME: 10.396982192993164 s | REWARD: 9.563281369160675 | FRAMES: 123 | QMAX: -39513330.0 | EPSILON: 0.5590000000000486 | CTE: 0.8122869587503252 | LOSS: 2.072367891218432e+16
EPISODE: 1261 | TIME: 8.948201656341553 s | REWARD: 6.539485340455125 | FRAMES: 98 | QMAX: 4043950000.0 | EPSILON: 0.5586500000000486 | CTE: 1.0341087992257143 | LOSS: 2.300267660627149e+16
EPISODE: 1262 | TIME: 10.615439176559448 s | REWARD: 13.69475467575902 | FRAMES: 124 | QMAX: 4845304300.0 | EPSILON: 0.5583000000000486 | CTE: 0.37157153457258063 | LOSS: 2.432704909934592e+16
EPISODE: 1263 | TIME: 12.902385711669922 s | REWARD: 18.613026121467243 | FRAMES: 142 | QMAX: 1397052200.0 | EPSILON: 0.5579500000000487 | CTE: 0.37473256770112673 | LOSS: 2.309669129289728e+16
EPISODE: 1264 | TIME: 11.387426614761353 s | REWARD: 13.453392333071402 | FRAMES: 145 | QMAX: 3234229200.0 | EPSILON: 0.5576000000000487 | CTE: 0.6764450583227585 | LOSS: 2.121277046049997e+16
EPISODE: 1265 | TIME: 9.392683744430542

EPISODE: 1303 | TIME: 6.435761213302612 s | REWARD: -0.10079068019119049 | FRAMES: 73 | QMAX: -823249660.0 | EPSILON: 0.5439500000000502 | CTE: 1.363385126524658 | LOSS: 1.9875266104918016e+16
EPISODE: 1304 | TIME: 13.29453730583191 s | REWARD: 11.329090819745074 | FRAMES: 151 | QMAX: 1451270100.0 | EPSILON: 0.5436000000000503 | CTE: 0.4859236071142847 | LOSS: 2.45319405142016e+16
EPISODE: 1305 | TIME: 13.150048017501831 s | REWARD: 13.288136243991032 | FRAMES: 155 | QMAX: 3779371500.0 | EPSILON: 0.5432500000000503 | CTE: 0.5334030099628387 | LOSS: 1.8927785582002176e+16
EPISODE: 1306 | TIME: 11.213093757629395 s | REWARD: 16.111869794562832 | FRAMES: 125 | QMAX: 4423037000.0 | EPSILON: 0.5429000000000503 | CTE: 0.48502004601608 | LOSS: 2.012543506500813e+16
EPISODE: 1307 | TIME: 7.675614595413208 s | REWARD: 2.310947294054744 | FRAMES: 89 | QMAX: 3325192700.0 | EPSILON: 0.5425500000000504 | CTE: 1.372600425572809 | LOSS: 2.184037040914432e+16
EPISODE: 1308 | TIME: 7.4739909172058105 s

EPISODE: 1346 | TIME: 10.954625368118286 s | REWARD: 8.779360760904389 | FRAMES: 132 | QMAX: 1294336.6 | EPSILON: 0.5289000000000519 | CTE: 0.3538057079561364 | LOSS: 2.3673952077348864e+16
EPISODE: 1347 | TIME: 8.151088237762451 s | REWARD: 2.954724062522054 | FRAMES: 86 | QMAX: 3119944200.0 | EPSILON: 0.5285500000000519 | CTE: 1.0178361899754653 | LOSS: 2.122325662315315e+16
EPISODE: 1348 | TIME: 13.694217920303345 s | REWARD: 14.638266370664311 | FRAMES: 150 | QMAX: -61250210.0 | EPSILON: 0.528200000000052 | CTE: 0.16995095919853326 | LOSS: 1.9132646932086784e+16
EPISODE: 1349 | TIME: 12.607181072235107 s | REWARD: 16.342429559647286 | FRAMES: 140 | QMAX: 3249249000.0 | EPSILON: 0.527850000000052 | CTE: 0.44601051809642855 | LOSS: 2.271708060844032e+16
EPISODE: 1350 | TIME: 8.411806344985962 s | REWARD: 6.836037097716716 | FRAMES: 88 | QMAX: 2406584300.0 | EPSILON: 0.527500000000052 | CTE: 0.9156397954170454 | LOSS: 1.9984594497437696e+16
EPISODE: 1351 | TIME: 17.11875605583191 s | 

EPISODE: 1389 | TIME: 11.543259143829346 s | REWARD: 14.120246642848329 | FRAMES: 132 | QMAX: 1488647200.0 | EPSILON: 0.5138500000000535 | CTE: 0.3244260916666667 | LOSS: 2.196293589087027e+16
EPISODE: 1390 | TIME: 13.93646788597107 s | REWARD: 12.144911665208818 | FRAMES: 162 | QMAX: 4601679400.0 | EPSILON: 0.5135000000000536 | CTE: 0.31829614942283957 | LOSS: 1.9590672981950464e+16
EPISODE: 1391 | TIME: 6.952749013900757 s | REWARD: 6.140208290545094 | FRAMES: 65 | QMAX: -12244098.0 | EPSILON: 0.5131500000000536 | CTE: 0.7792595640153845 | LOSS: 2.1474776350457856e+16
EPISODE: 1392 | TIME: 15.345268964767456 s | REWARD: 4.7586892205595746 | FRAMES: 176 | QMAX: 1818691000.0 | EPSILON: 0.5128000000000537 | CTE: -1.0129000493757387 | LOSS: 1.913071634428723e+16
EPISODE: 1393 | TIME: 17.262232303619385 s | REWARD: 9.941017205974804 | FRAMES: 201 | QMAX: 1941749800.0 | EPSILON: 0.5124500000000537 | CTE: -0.3744389433044776 | LOSS: 1.8404336442802176e+16
EPISODE: 1394 | TIME: 7.26911902427

KeyboardInterrupt: 